In [3]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.io
import datetime
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import normalize
import loader

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
train_dict = loader.load_training_data()
print(train_dict.keys())

dict_keys(['XVal', 'yVal', 'X', 'y'])


In [3]:
X = train_dict['X']
y = train_dict['y']

In [12]:
print(X[0])
print(y[0])

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

In [5]:
# from sklearn.datasets import make_classification

# X, y = make_classification(500, n_features=2, n_informative=2, n_redundant=0, n_repeated=0)
# pd.DataFrame({"x1":X[:,0], "x2":X[:,1],"y":y}).plot(kind='scatter',x='x1', y='x2', c='y')
# y = pd.get_dummies(y).values


In [6]:
def add_bias(in_val):
    return np.vstack([in_val, np.ones(len(in_val[0]))])

def initialize_layer(input_dim, output_dim, seed):
    np.random.seed(seed)
    z = np.random.normal(0,0.01,output_dim)
    y = np.random.normal(0,0.01,output_dim)
    w = np.random.normal(0,0.01,(input_dim, output_dim))
    b = np.ones(output_dim)
    return z, y, w, b

In [10]:
def squared_loss_raw(y,y_2): return np.power((y - y_2), 2)
def squared_loss_deriv_raw(y, y_2): return (y_2 - y)
squared_loss = np.vectorize(squared_loss_raw)
squared_loss_deriv = np.vectorize(squared_loss_deriv_raw)

def calc_dEdz2(y_2, y, cost_deriv):
    return np.diag(
        np.multiply(y_2, np.subtract(1, y_2))
    ).dot(cost_deriv(y, y_2))

def calc_dEdz1(z_1, w_2, dEdz2):
    return np.diag(
        np.subtract(1, np.power(np.tanh(z_1), 2))
    ).dot(w_2).dot(dEdz2)

def predictNN(seq, X):
    temp = X
    for comm in seq:
        if callable(comm):
            temp = comm(temp)
        else:
            temp = temp.dot(comm)
    return temp.argmax(axis=1)

def scoreNN(seq, X, y):
    preds = predictNN(seq, X)
    return sum(preds != y) / len(preds)

In [11]:
params = {
    "hidden_layer_size": 200,
    "eta":1,
    "epochs": 30,
    "seed": 20,
    "epoch_length": 10000,
    
    "cost_func": squared_loss,
    "cost_func_deriv": squared_loss_deriv
}

def fix_params(params, X):
    if params.get("epoch_length", 0) == 0:
        params['epoch_length'] = len(X)
        
    if params.get("seed", 0) == 0:
        params['seed'] = 100    
    return params

def trainNN(X, y, params):
    params = fix_params(params, X)

    hl_size = params['hidden_layer_size']
    eta = params['eta']
    epoch_length = params['epoch_length']
    seed = params['seed']
    feature_dim = len(X[0])
    output_dim = 10
    start_time = datetime.datetime.now()
    last = start_time
    training_errors = []
    validation_errors = []
    
    cost = params['cost_func']
    cost_deriv = params['cost_func_deriv']

    z_1, y_1, w_1, b_1 = initialize_layer(feature_dim, hl_size, seed)
    z_2, y_2, w_2, b_2 = initialize_layer(hl_size, output_dim, seed)
    
    dEdw2 = np.zeros((hl_size, output_dim))
    dEdw1 = np.zeros((feature_dim, hl_size))
    
    print("-"*25, "Initialization") 
    print("Start Time:", start_time.isoformat())
    for k, v in params.items():
        if "func" not in str(v): print(k, v)
    print("-"*25, "Training")
    for epoch in range(params['epochs']):

        for _ in range(epoch_length):
            training_point = int(np.random.choice(len(y), 1))
            y_0, truth = X[training_point], y[training_point]
            
            # forward pass
            y_0.dot(w_1, out=z_1)
            z_1 += b_1 # add bias
            np.tanh(z_1, out=y_1)
            
            y_1.dot(w_2, out=z_2)
            z_2 += b_2 # add bias
            g = lambda z_2: 1 / (1 + np.exp(np.multiply(-1, z_2)))
            y_2 = g(z_2)
            

            # cache
            dEdz2 = calc_dEdz2(y_2, truth, cost_deriv)
            dEdz1 = calc_dEdz1(z_1, w_2, dEdz2)

            # backprop
#             print(dEdz2.shape)
#             print(dEdz1.shape)
            y_1.reshape(200,1).dot(dEdz2.reshape(1,10), out=dEdw2)
            y_0.reshape(784,1).dot(dEdz1.reshape(1,200), out=dEdw1)

            # updates
            w_1 -= eta * dEdw1
            w_2 -= eta * dEdw2
#             b_1 -= eta * np.sum(dEdz1, axis=0)
#             b_2 -= eta * np.sum(dEdz2, axis=0)
            
   
        prediction_sequence = [w_1, lambda x: x + b_1, np.tanh, 
                               w_2, lambda x: x + b_2, np.vectorize(lambda x: 1 / 1 + np.exp(-1 * x))]
        print("Epoch", epoch, "\tEpoch Train Time: %.2f" % (datetime.datetime.now() - last).total_seconds())
        last = datetime.datetime.now()
        train_score = scoreNN(prediction_sequence, X, y)
        print("Training Error: %.5f" % train_score)
#         validation_score = scoreNN(prediction_sequence, Xt, yt)
#         print("Validation Error: %.5f" % validation_score)
        if train_score > 0.99:
            print("Error too high")
            break
        print("-"*20)

    
    end_time = datetime.datetime.now()
    print("End Time:", end_time.isoformat())
    print("Total Seconds: %.2f" % (end_time - start_time).total_seconds())
    return prediction_sequence

seq = trainNN(X, y, params)

------------------------- Initialization
Start Time: 2015-11-19T16:46:32.603486
epoch_length 10000
hidden_layer_size 200
eta 1
epochs 30
seed 20
------------------------- Training
Epoch 0 	Epoch Train Time: 14.81
Training Error: 0.90105
--------------------
Epoch 1 	Epoch Train Time: 18.95
Training Error: 0.90105
--------------------
Epoch 2 	Epoch Train Time: 18.34
Training Error: 0.90105
--------------------
Epoch 3 	Epoch Train Time: 21.01
Training Error: 0.90105
--------------------


KeyboardInterrupt: 

In [ ]:
kaggle_out = pd.DataFrame(predictNN(seq, kaggleTest)).reset_index()
kaggle_out.columns = ['Id', 'Category']
kaggle_out.Id = kaggle_out.Id + 1
t = str(datetime.datetime.now())
fname = "Kaggle prediction " + t + ".csv"
print(fname)

In [ ]:
kaggle_out.to_csv(fname, index=None)

In [4]:
def squared_error(y, y_hat):
    return (y - y_hat).dot((y - y_hat))

def squared_error_der(y, y_hat):
    return -1 * (y - y_hat)

def t(z):
    return np.tanh(z)

def t_der(z):
    return 1 - np.power(t(z), 2)

def g(z):
    return (1 / (1 + np.exp(-1 * z)))

def g_der(z):
    return g(z) * (1 - g(z))

def dEdz2(z_2, y_2, y, loss_der):
    return np.multiply(g_der(z_2), loss_der(y, y_2))

def dEdz1(z_1, w_2, dEdz2_cached):
    return np.diag(t_der(z_1)).dot(w_2).dot(dEdz2_cached)

def l2_deriv(y_1, dEdz2_cached):
    return np.outer(y_1, dEdz2_cached.T)

def l1_deriv(y_0, dEdz1_cached):
    return np.outer(y_0, dEdz1_cached.T)

In [8]:
g_der(5)

0.0066480566707900332